In [309]:
import json
import requests
from datetime import datetime

ELASTIC = "http://localhost:9200" #dirección en la que está funcionando elastic

In [239]:
"""
    Media, mejor y peor tiempo por grupo de edad:
"""

query = {
    "size" : 0,
    "aggs" : {
        "tiempos_por_grupo" : {
            "terms" : {
                "field" : "Group.keyword"
            },
            "aggs" : {
                "tiempos" : {
                    "avg" : {
                        "field" : "Time"
                    }
                },
                "mejor-tiempo" : {
                    "min" : {
                        "field" : "Time"
                    }
                },
                "peor-tiempo" : {
                    "max" : {
                        "field" : "Time"
                    }
                }
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["aggregations"]["tiempos_por_grupo"]["buckets"]

for r in res:
    
    print("Grupo de edad %s: %d corredores, tiempo medio %s, \
tiempo perdedor: %s, tiempo ganador: %s" % (r["key"], r["doc_count"],
                                                r["tiempos"]["value_as_string"],
                                                r["peor-tiempo"]["value_as_string"],
                                                r["mejor-tiempo"]["value_as_string"]))

Grupo de edad 20-39: 272 corredores, tiempo medio 1:47:28, tiempo perdedor: 2:41:04, tiempo ganador: 1:09:47
Grupo de edad 40-49: 139 corredores, tiempo medio 1:46:51, tiempo perdedor: 2:37:23, tiempo ganador: 1:18:36
Grupo de edad 50-59: 38 corredores, tiempo medio 1:51:09, tiempo perdedor: 2:27:23, tiempo ganador: 1:26:51
Grupo de edad 60-98: 7 corredores, tiempo medio 1:54:26, tiempo perdedor: 2:05:21, tiempo ganador: 1:37:56
Grupo de edad 01-19: 5 corredores, tiempo medio 1:44:14, tiempo perdedor: 2:04:02, tiempo ganador: 1:21:24
Grupo de edad 99-+: 2 corredores, tiempo medio 2:01:10, tiempo perdedor: 2:22:11, tiempo ganador: 1:40:09


In [310]:
"""
    Mejor posición por cada Estado
"""

query = {
    "size" : 0,
    "aggs" : {
        "estados" : {
            "terms" : {
                "field" : "State.keyword"
            },
            "aggs" : {
                "mejor" : {
                    "min" : {
                        "field" : "Place"
                    }
                },
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["aggregations"]["estados"]["buckets"]

for r in res:
    print("%s: %d corredores, mejor posición %d" % (r["key"], r["doc_count"], r["mejor"]["value"]))

RI: 190 corredores, mejor posición 3
MA: 182 corredores, mejor posición 1
CT: 32 corredores, mejor posición 20
NH: 20 corredores, mejor posición 12
FL: 9 corredores, mejor posición 17
NC: 6 corredores, mejor posición 54
MO: 5 corredores, mejor posición 7
IN: 4 corredores, mejor posición 58
NJ: 4 corredores, mejor posición 69
PA: 4 corredores, mejor posición 100


In [330]:
"""
    Corredores con Pace entre 6 y 8 minutos que hayan quedado por encima del puesto 5 en su grupo
"""

query = {
    "size": 1000,
    "query" : {
        "bool" : {
            "must" : [{
                "range" : { 
                    "Pace" : {
                        "gte" : "0:6:00",
                        "lte" : "0:8:00"
                    }
                }},
                {"range" : {
                    "GroupPlace" : {
                        "lte" : 5
                    }
                }
            }]
        }
    }
}


res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["hits"]["hits"]

for re in res:
    r = re["_source"]
    print("%s, Pace: %s, Puesto en su grupo(%s) %d" % (r["FirstName"]+" "+r["LasName"], r["Pace"], r["Group"], r["GroupPlace"]))

CRUZ TAILOR, Pace: 0:6:00, Puesto en su grupo(40-49) 1
BRENTON FAGO, Pace: 0:6:03, Puesto en su grupo(40-49) 2
CHARLES FARLESS, Pace: 0:6:03, Puesto en su grupo(40-49) 3
WILBER VANORDER, Pace: 0:6:04, Puesto en su grupo(40-49) 4
KENDRICK HOLZ, Pace: 0:6:13, Puesto en su grupo(01-19) 1
JAMA PEAD, Pace: 0:6:16, Puesto en su grupo(20-39) 1
ROMEO UNVARSKY, Pace: 0:6:18, Puesto en su grupo(40-49) 5
ALEXIS HABERMANN, Pace: 0:6:38, Puesto en su grupo(50-59) 1
MAGGIE NASR, Pace: 0:6:39, Puesto en su grupo(20-39) 2
FRANCIS LAMSON, Pace: 0:6:42, Puesto en su grupo(50-59) 2
HEATH KIRVIN, Pace: 0:7:02, Puesto en su grupo(50-59) 3
NEDRA NASSIMI, Pace: 0:7:03, Puesto en su grupo(20-39) 3
FLORIDA AYTES, Pace: 0:7:03, Puesto en su grupo(40-49) 1
LETTIE MCKAGUE, Pace: 0:7:05, Puesto en su grupo(20-39) 4
MAIRA ROUTHIER, Pace: 0:7:11, Puesto en su grupo(20-39) 5
GIOVANNI FYLES, Pace: 0:7:17, Puesto en su grupo(50-59) 4
STEPHANI FERTITTA, Pace: 0:7:18, Puesto en su grupo(40-49) 2
WESLEY BECKWORTH, Pace: 0

In [332]:
"""
    Número de BIB de los diez mejores corredores
"""

query = {
    "size": 10,
    "query" : {
        "range" : {
            "Place" : {
                "lte" : "10"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["hits"]["hits"]

for re in res:
    r = re["_source"]
    print("%s, puesto %s, BIB: %s" % (r["FirstName"]+" "+r["LasName"], r["Place"], r["BIBNumber"]))

ANDRE PELLAM, puesto 1, BIB: 340
HORACE KARPOWICZ, puesto 2, BIB: 34
MILES ROSELLA, puesto 3, BIB: 399
FRANKLYN GIRARDIN, puesto 4, BIB: 18
AL LEMASTER, puesto 5, BIB: 262
CRUZ TAILOR, puesto 6, BIB: 308
DEWEY HEAIVILIN, puesto 7, BIB: 45
HYMAN CURIE, puesto 8, BIB: 16
BRENTON FAGO, puesto 9, BIB: 42
CHARLES FARLESS, puesto 10, BIB: 420
